2025-10-28 23:38:55 Tuesday


面试问题记录

## 1.bert的输出

bert的输出有两类结果

| 输出                       | 形状                                   | 含义                                       |
| ------------------------ | ------------------------------------ | ---------------------------------------- |
| **1. last_hidden_state** | `[batch_size, seq_len, hidden_size]` | 每个 token 的上下文向量表示                        |
| **2. pooler_output**     | `[batch_size, hidden_size]`          | [CLS] 位置经过一个 `tanh` 层后的句向量（BERT 的默认句子表示） |


下面是一个代码实现，注意看结尾注释


In [4]:
import torch
from transformers import BertTokenizer, BertModel

# 1. 加载预训练的中文 or 英文 BERT
# 如果你主要是英文文本，用 'bert-base-uncased'
# 如果你想中文，用 'bert-base-chinese'
MODEL_NAME = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

# 2. 准备输入文本（单句 or 句对都行）
sentence_a = "Hello world."
sentence_b = "How are you today?"
# sentence_c = "How are you today?"


# 3. tokenizer：把文本 → token ids / segment ids / attention mask
encoded = tokenizer(
    sentence_a,
    sentence_b,                    # 传第二个句子 -> 会自动加 [SEP] 并生成 token_type_ids区分句A/B
    # sentence_c,
    
    padding=True,
    truncation=True,
    return_tensors="pt",           # 直接返回 PyTorch tensor
)

print("=== tokenizer 输出张量 ===")
for k, v in encoded.items():
    print(k, v.shape, v)

# encoded 里典型有：
# - input_ids:        token id 序列
# - token_type_ids:   句子A是0，句子B是1
# - attention_mask:   1=有效token，0=padding

# 4. 前向：送进 BERT
with torch.no_grad():
    outputs = model(**encoded)

# 5. 拿结果
last_hidden_state = outputs.last_hidden_state    # [batch, seq_len, hidden]
pooler_output    = outputs.pooler_output         # [batch, hidden]，等价于"句子向量"

print("\n=== BERT 输出 ===")
print("last_hidden_state:", last_hidden_state.shape)
print("pooler_output:", pooler_output.shape)

# 6. 看看 [CLS] 向量，和任意一个 token 的向量
cls_embedding = last_hidden_state[:, 0, :]       # 第0个位置是 [CLS]
tok1_embedding = last_hidden_state[:, 1, :]      # 第1个位置是第一个真实token

print("\n[CLS] embedding shape:", cls_embedding.shape)
print("First real token embedding shape:", tok1_embedding.shape)

# 7. 举个实际数值片段
print("\n[CLS] embedding (前10维):")
print(cls_embedding[0, :10])

print("\n第一个token的embedding (前10维):")
print(tok1_embedding[0, :10])


##########################################
# 📌 关键解读（看看打印出来的东西时可以对照下面读）
##########################################

# 1) input_ids
#    这是 tokenizer 把 "[CLS] Hello world . [SEP] How are you today ? [SEP]"
#    全部换成词表ID后的整数序列。
#
# 2) token_type_ids
#    同长度向量，比如 [0,0,0,0,0,1,1,1,1,1,1,...]
#    0 表示属于句子A，1 表示属于句子B。
#
# 3) attention_mask
#    和 seq_len 等长，比如 [1,1,1,1,1,1,...,1,0,0,0]
#    告诉BERT哪些位置是真实token(1)，哪些是padding(0)，
#    注意力会在padding上被屏蔽，不会影响别的词。
#
# 4) last_hidden_state
#    形状 [batch_size, seq_len, hidden_size]
#    每个token一个向量（768维 for bert-base）。
#    适合做 token-level 任务，例如 NER、问答里的span抽取。
#
# 5) pooler_output
#    形状 [batch_size, hidden_size]
#    这是 BERT 官方"句子向量"：它取 last_hidden_state[:,0,:] 也就是 [CLS]，
#    但还过了一层线性+Tanh。常拿来做句子分类。
#
# 6) cls_embedding
#    我们手动取的 last_hidden_state[:,0,:]。
#    跟 pooler_output 很接近，但 pooler_output 还多一层变换。
#
# ✅ 总结:
# - 如果你要句子分类，用 pooler_output 或 [CLS] 向量接线性层。
# - 如果你要逐词标注，用 last_hidden_state 的每个位置。


=== tokenizer 输出张量 ===
input_ids torch.Size([1, 11]) tensor([[ 101, 7592, 2088, 1012,  102, 2129, 2024, 2017, 2651, 1029,  102]])
token_type_ids torch.Size([1, 11]) tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])
attention_mask torch.Size([1, 11]) tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

=== BERT 输出 ===
last_hidden_state: torch.Size([1, 11, 768])
pooler_output: torch.Size([1, 768])

[CLS] embedding shape: torch.Size([1, 768])
First real token embedding shape: torch.Size([1, 768])

[CLS] embedding (前10维):
tensor([-0.2158,  0.2033, -0.0384, -0.3991, -0.4776, -0.2867,  0.7173,  0.7658,
        -0.0283, -0.3685])

第一个token的embedding (前10维):
tensor([ 0.0255,  0.0870,  0.7774, -0.7172,  0.2613, -0.0265,  0.5924,  0.6962,
        -0.6278, -1.4433])


## 2. BERT 预训练阶段的两个任务

### 2.1 Masked Language Modeling (MLM)

给定被随机掩蔽的输入序列，模型需要预测被掩蔽 token 的原始词表 ID。  
与传统的单向语言模型（仅依赖左或右上下文）不同，BERT 通过 **双向 self-attention** 同时利用左、右两侧上下文信息。  
损失函数为掩蔽位置处的交叉熵损失：

$$
\mathcal{L}_{\text{MLM}} = -\sum_{i \in M} \log p_\theta(x_i \mid x_{\setminus i})
$$

---

### 2.2 Next Sentence Prediction (NSP)

该任务用于学习句子间关系。输入由句子 A 和句子 B 组成，其中 50% 的样本中 B 为 A 的真实下一句（IsNext），其余 50% 为随机句（NotNext）。  
模型使用 `[CLS]` token 的输出向量，经线性层和 softmax，预测两者是否连续。对应的损失为二分类交叉熵：

$$
\mathcal{L}_{\text{NSP}} = -\log p_\theta(y \mid \text{[CLS]})
$$

---

### 2.3 总体损失

预训练阶段优化以下总损失函数：

$$
\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{MLM}} + \mathcal{L}_{\text{NSP}}
$$
